In [ ]:
from new_helper import *

# DIRECTORY_PATH = "upload.old/"

# upload_documents_in_directory(DIRECTORY_PATH)


# Example usage:
directory_path = "upload"
process_files(directory_path)

print(file_cid, metadata_cid) 


In [ ]:
#Option 1
# Example usage

from new_helper import *

directory_path = "upload"
for file_cid, metadata_cid in process_files(directory_path):
    # Now you can use the returned values here
    print("File CID:", file_cid)
    print("Metadata CID:", metadata_cid)


In [ ]:
#Option 3
from new_helper import *

directory_path = "upload"

# Call the function and store its result
result = process_files(directory_path)

# Now you can access the file and metadata CIDs as a list of tuples
file_cid, metadata_cid = result[0]  # Access the first tuple in the list

# Pass a list
value1 = encoder([file_cid, metadata_cid])
print(value1)

# Pass a dictionary
cid_dict = process_files(directory_path)
value2 = encoder(list(cid_dict.values()))
print(value2)

In [1]:
#Option 4

from new_helper import *

directory_path = "upload"

# project_account_id = {'account_id': '09938@test'}
user_details = process_files(directory_path)

# # Assuming cid_dict is the output from process_files(), you would do something like:
# project_account_id.update({username: user_details[filename]})

2024-11-17 15:04:48.543 | INFO     | new_helper:parse_file_with_tika:29 - Parsing upload/Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf with Tika...


file:  upload/Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf
file metadata:  {'pdf:PDFVersion': '1.7', 'pdf:docinfo:title': 'Aeroacoustic airfoil shape optimization enhanced by autoencoders', 'xmp:CreatorTool': 'Elsevier', 'pdf:hasXFA': 'false', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dc:format': 'application/pdf; version=1.7', 'pdf:docinfo:custom:robots': 'noindex', 'pdf:docinfo:creator_tool': 'Elsevier', 'access_permission:fill_in_form': 'true', 'pdf:docinfo:custom:CreationDate--Text': '7th February 2023', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'dc:title': 'Aeroacoustic airfoil shape optimization enhanced by autoencoders', 'pdf:containsNonEmbeddedFont': 'false', 'pdf:docinfo:custom:CrossMarkDomains[1]': 'elsevier.com', 'robots': 'noindex'

In [ ]:
from Crypto.Hash import keccak
import os
import binascii
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import sys
import json
from ipfs_functions import *
import logging

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]

iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))

# Function to link details using blockchain
def set_account_detail(address, account, key, value):
    params = integration_helpers.get_first_four_bytes_of_keccak(
        b"setAccountDetail(string,string,string)"
    )
    no_of_param = 3
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(account)  # account id
    params = params + integration_helpers.argument_encoding(key)  # key
    params = params + integration_helpers.argument_encoding(value)  # value
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash


def encoder():
    encoded_value = ''.join(
        integration_helpers.argument_encoding(v).decode('utf-8') 
        for v in (file_cid, metadata_cid)
     )
    
    # Return the string value
    return encoded_value
                    
value = encoder()

print(value)

# Call the function and print its result
print("Some function output:", encoder())

hash = set_account_detail(
    address,
    project_account['account_id'],
    "key",
    value
)